In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection

In [11]:
cust_data = pd.read_csv('data_transform.csv')
cust_data.shape

(100233, 126)

### 필요없는 컬럼 삭제

In [12]:
cust_data = cust_data.drop(['MON_TLFE_AMT','SVINS_MON_PREM','MAX_MON_PREM','STLN_REMN_AMT','GDINS_MON_PREM','AVG_STLN_RATE',
                            'FMLY_SVINS_MNPREM','CRLN_OVDU_RATE','FMLY_GDINS_MNPREM','TOT_CRLN_AMT','MIN_CNTT_DATE','ARPU',
                            'CRDT_LOAN_CNT','LT1Y_STLN_AMT','TOT_REPY_AMT','CUST_JOB_INCM','FMLY_CLAM_CNT','TEL_CNTT_QTR',
                            'ACTL_FMLY_NUM','CUST_FMLY_NUM','HSHD_INFR_INCM','MATE_JOB_INCM','FMLY_TOT_PREM','FYCM_PAID_AMT',
                            'AVG_CALL_FREQ','TOT_PREM'
                           ], axis=1)

In [13]:
# selected_var = ['TARGET','OVDU_HIGH_RATE','CRDT_CARD_CNT','AUTR_FAIL_MCNT','MATE_OCCP_NAME_C_02','MATE_OCCP_NAME_C_03',
#                 'MATE_OCCP_NAME_C_09','MATE_OCCP_NAME_C_13','MATE_OCCP_NAME_C_16']
# cust_l1_data = cust_data[selected_var]
# cust_l1_data.head()

In [14]:
def model_performance(y_test, y_pred):    
    print('confusion matrix')
    print(metrics.confusion_matrix(y_test, y_pred))
    print('accuracy : {}'.format(metrics.accuracy_score(y_test, y_pred).round(3)))
    print('precision : {}'.format(metrics.precision_score(y_test, y_pred, pos_label=1).round(3)))
    print('recall : {}'.format(metrics.recall_score(y_test, y_pred, pos_label=1).round(3)))
    print('F1 : {}'.format(metrics.f1_score(y_test, y_pred, pos_label=1).round(3)))

### ML modeling

In [15]:
# train / test set 분리
x = cust_data.drop('TARGET', axis=1)
y = cust_data['TARGET']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

print('연체자 수 train : {} / {}'.format(sum(y_train == 1), y_train.shape[0]))
print('연체자 수 test  : {} / {}'.format(sum(y_test  == 1), y_test.shape[0]))

연체자 수 train : 3439 / 80186
연체자 수 test  : 848 / 20047


In [16]:
# train set 중에서 연체자만 추출
x_overdue = x_train[y_train == 1]
y_overdue = y_train[y_train == 1]
print(x_overdue.shape)
print(y_overdue.shape)

(3439, 99)
(3439,)


In [17]:
clf = GradientBoostingClassifier(n_estimators=500)
clf_0 = GradientBoostingClassifier(n_estimators=500)
clf_1 = GradientBoostingClassifier(n_estimators=500)

In [19]:
max_epoch = 1

for i in range(max_epoch+1):
    x2_train, x2_test, y2_train, y2_test = train_test_split(x_train, y_train, test_size=0.2, random_state=0)
    
    print('========================================= ', i)
    
    clf.fit(x2_train, y2_train)

    y_pred_f = clf.predict(x_test)
    model_performance(y_test, y_pred_f)

    # 연체자 duplicate
    x_train = pd.concat([x_train, x_overdue])
    y_train = pd.concat([y_train, y_overdue])

    if i == max_epoch :
        y_pred = clf.predict(x_train)

        x3_train_0 = x_train[y_pred == 0]
        y3_train_0 = y_train[y_pred == 0]

        x3_train_1 = x_train[y_pred == 1]
        y3_train_1 = y_train[y_pred == 1]

        clf_0.fit(x3_train_0, y3_train_0)
        clf_1.fit(x3_train_1, y3_train_1)

        y_pred = clf.predict(x_test)
        model_performance(y_test, y_pred)

        y_pred_0 = clf_0.predict(x_test)
        model_performance(y_test, y_pred_0)

        y_pred_1 = clf_1.predict(x_test)
        model_performance(y_test, y_pred_1)

        print(y_pred.shape[0])
        print(y_pred_0.shape[0])
        print(y_pred_1.shape[0])

=========================================  0
confusion matrix
[[18395   804]
 [  372   476]]
accuracy : 0.941
precision : 0.372
recall : 0.561
F1 : 0.447
=========================================  1
confusion matrix
[[18262   937]
 [  340   508]]
accuracy : 0.936
precision : 0.352
recall : 0.599
F1 : 0.443
confusion matrix
[[18262   937]
 [  340   508]]
accuracy : 0.936
precision : 0.352
recall : 0.599
F1 : 0.443
confusion matrix
[[18917   282]
 [  656   192]]
accuracy : 0.953
precision : 0.405
recall : 0.226
F1 : 0.29
confusion matrix
[[ 3239 15960]
 [   64   784]]
accuracy : 0.201
precision : 0.047
recall : 0.925
F1 : 0.089
20047
20047
20047
